In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd 
import dataretrieval.nwis as nwis
from datetime import date, timedelta
import pylab
from scipy import stats
from pylr2 import regress2
import pymannkendall as mk

In [3]:
home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"
sbr_ref_combo_diss = gpd.read_file(os.path.join(home, "Data", "Catchments", "Reference", "gages_ii","reference_keep.shp"))

In [12]:
def create_recession_df(gage, w, C):
    # gage= usgs gage id 
    # w is the streamflow precision thredhold 
    # C is a scalar that we multiply by w. C >= 1
    
    site = gage

    # Download the streamflow data
    streamflow_df = nwis.get_record(sites=site, service = 'dv', start = '1984-01-01', end = '2021-12-31', parameterCd = '00060') # mean daily ft3/s 
    streamflow_df.reset_index(inplace=True)
    # commented out below because we will use estimated values as well for gap-free time series 
    # also only using gages with no missing data, so no need to drop na's 
    #streamflow = streamflow_df[streamflow_df['00060_Mean_cd'] == 'A']
    #streamflow_df.dropna(subset=['00060_Mean'])
    #streamflow = streamflow.reset_index(drop=True)
    streamflow = streamflow_df[["datetime", "00060_Mean"]]
    streamflow.columns = ["Date", "originalQ"] # mean daily ft3/s 

    # Convert from ft3/s to mm/day 
    area_m2 = sbr_ref_combo_diss[sbr_ref_combo_diss['GAGE_ID'] == site].AREA.iloc[0] # m2 
    m2_to_ft2 = 10.7639
    ft_to_mm = 304.8
    s_to_day = 60*60*24
    convert= ((ft_to_mm * s_to_day) / (area_m2 * m2_to_ft2))
    streamflow['originalQ'] = streamflow['originalQ'].multiply(convert)

   
    # calculate dQ and Q 
    streamflow['dQ'] = streamflow['originalQ'].diff()
    # Calculate Q = Qi + Qi-1 / 2 (matches dQ time steps)
    streamflow['Q'] = (streamflow['originalQ'] + (streamflow['originalQ'] + streamflow['dQ'].multiply(-1)))/2
    streamflow = streamflow.reset_index(drop=True)

    # Calculate time-scaled dQ and Q 
    dQ_ts = [np.nan]
    Q_ts = [np.nan]
    for i in range(1, streamflow.shape[0]):
        q_diff = streamflow['originalQ'].iloc[i] - streamflow['originalQ'].iloc[i-1]
        q = np.sum(streamflow['originalQ'].iloc[(i-1):(i +1)])/2
        if abs(q_diff) >= (w*C):
            dQ_ts.append(q_diff)
            Q_ts.append(q)
            continue
    
        steps = 1
        if abs(q_diff) < (w*C):
            for j in range(0, i):
                steps +=1
                q_diff_again = streamflow['originalQ'].iloc[i] - streamflow['originalQ'].iloc[i-steps]
                if abs(q_diff_again) < (w*C):
                    continue
                else: 
                    #dQ_ts.append(q_diff_again/(steps))
                    #Q_ts.append(np.sum(streamflow['originalQ'].iloc[(i-steps):(i+1)])/(steps))
                    date_i = streamflow['Date'].iloc[i]
                    date_old = streamflow['Date'].iloc[(i-steps)]
                    date_diff = date_i - date_old
                    delta = date_diff.days
                    dQ_ts.append(q_diff_again/delta)
                    Q_ts.append(np.sum(streamflow['originalQ'].iloc[(i-steps):(i+1)])/(delta))
                    break
    
    streamflow['dQ_ts'] = dQ_ts
    streamflow['Q_ts'] = Q_ts

    # calculate absolute value of change in dQ
    streamflow['dQdQ'] = streamflow['dQ_ts'].abs().diff()

    # drop non-consecutive dates because that will have an incorrect difference 
    streamflow['day_before'] = pd.concat([pd.Series([np.nan]), streamflow['Date'].iloc[0:(streamflow.shape[0]-1)].add(timedelta(days=1))]).to_list()
    streamflow = streamflow[(streamflow['Date'] == streamflow['day_before'])]
    streamflow = streamflow.drop(['day_before'], axis=1)

    # identify recession events 
    days_to_remove = ( (streamflow['dQ_ts'] >= 0) |   # removes days of non decreasing flow - switch to the threshold
                    (streamflow['Q_ts'] <= 0) |   # remove days of 0 streamflow
                    (streamflow['dQdQ'] >= 0)  # remove days where the absolute value of the derivate is not decreasing 
                    )
    decreasing_streamflow = streamflow[-days_to_remove]   

    # Individual events must be at least 5 consecutive days 
    # Remove the first day of each event
    # loop through and increase the counter for every time there is a consevutive date and put that counter in the index for every loop 
    counting_days = []
    consecutive = 0
    for i in range(0, (decreasing_streamflow.shape[0]-1)):
        if i == 0:
            date = decreasing_streamflow['Date'].iloc[i]
            date_next = decreasing_streamflow['Date'].iloc[i+1]
            if (date+timedelta(days=1)) == date_next:
                consecutive += 1
            else: 
                consecutive = 0
            counting_days.append(consecutive)
    
        if i >0:
            date = decreasing_streamflow['Date'].iloc[i]
            date_next = decreasing_streamflow['Date'].iloc[i+1]
            date_prior = decreasing_streamflow['Date'].iloc[i-1]

            if (date+timedelta(days=1)) == date_next:
                consecutive += 1
                counting_days.append(consecutive)  
            elif ( (date+timedelta(days=1) != date_next) & (date-timedelta(days=1) == date_prior)):
                consecutive += 1
                counting_days.append(consecutive)  
                consecutive = 0
            else: 
                consecutive =0
                counting_days.append(consecutive)      

    decreasing_streamflow['consec_days'] = counting_days + [np.nan]   
    decreasing_streamflow = decreasing_streamflow.reset_index(drop=True)

    # find the events that are at least 5 days long 
    idx_gte5 = decreasing_streamflow[decreasing_streamflow['consec_days']>=5].index.tolist()
    idx_1 = decreasing_streamflow[decreasing_streamflow['consec_days']==1].index.tolist()

    K = 0
    for upper_idx in idx_gte5:
        K += 1
        met_condition = [j for j in idx_1 if j<upper_idx ]
        lower_idx = met_condition[len(met_condition)-1]
        if K == 1:
            keep_idx = list(range(lower_idx, upper_idx+1))
        else:
            keep_idx = keep_idx + list(range(lower_idx, upper_idx+1))
    
    final_idx = np.unique(keep_idx)
    recessions = decreasing_streamflow.iloc[final_idx]
    recessions = recessions.reset_index(drop=True)

    # drop the first day of each recession 
    recessions = recessions[recessions['consec_days'] != 1]

    # subset to the months of interest (june - august)
    recessions = recessions.loc[recessions['Date'].dt.month.isin([5,6,7,8,9])]

    # label each individual event
    event_number = []
    event_counter = 0
    for i in range(0, recessions.shape[0]):
        if i == 0:
            event_counter = 1
            event_number.append(event_counter)
        else: 
            if recessions['consec_days'].iloc[i] > recessions['consec_days'].iloc[i-1]:
                event_counter = event_counter + 0
                event_number.append(event_counter)
            else:
                event_counter = event_counter + 1
                event_number.append(event_counter)

    recessions['event_number'] = event_number

    # take the log transform of Q and dQ 
    recessions['log_dQ']=np.log10(recessions['dQ_ts'].abs()) # make sure to take the log of the absolute value of dQ
    recessions['log_Q']=np.log10(recessions['Q_ts'])
    
    return recessions



def calculate_coef(df, gage, reg_type):
    start_year = df.Date.iloc[0].year
    if reg_type == "OLS":
        slope, intercept, r_value, p_value, std_err = stats.linregress(df.log_Q, df.log_dQ)
        b_val = slope
        medEvSlope = np.median(b_val)
        df['log_dQ_offset'] = df.log_dQ - (df.log_Q * medEvSlope)
        a_val = np.mean(df.log_dQ_offset)
        return {'Gage':gage, 'Year':start_year, 'A':a_val, 'B':b_val}
    if reg_type == "RMA":
        results = regress2(df.log_Q, df.log_dQ, _method_type_2="reduced major axis")
        b_val = results['slope']
        medEvSlope = np.median(b_val)
        df['log_dQ_offset'] = df.log_dQ - (df.log_Q * medEvSlope)
        a_val = np.mean(df.log_dQ_offset)
        return {'Gage': gage, 'Year':start_year, 'A':a_val, 'B':b_val}



def rolling(df, gage, reg_type, interval):

    # make a list of start and end years for each interval 
    start_year = df.Date.iloc[0].year
    end_year = df.Date.iloc[df.shape[0]-1].year
    starts = range(start_year, end_year + 2 - interval)
    ends = [x + (interval-1) for x in starts]

    # loop through each interval 
    coefficients = []
    for i in range(0, len(starts)):
        start = starts[i]
        end = ends[i]
        df_sub = df.loc[df['Date'].dt.year.isin(range(start, end + 1))]
        if df_sub.shape[0] == 0: 
            continue
        df_sub = df_sub.reset_index(drop=True)
        coefficients.append(calculate_coef(df_sub, gage, reg_type))
    
    return pd.DataFrame(coefficients)


In [11]:
# first subset the reference watersheds to those with no missing data 
def check_streamflow(site):
    
    # quality control 
    # A = Approved for publication 
    # P = Provisional data subject to revision 
    # e = Value has been estimated 

    streamflow_df = nwis.get_record(sites=site, service = 'dv', start = '1900-01-01', parameterCd = '00060')
    if streamflow_df.shape[0] == 0:
        return np.nan
    date_index = streamflow_df.index.to_series().between('1984-01-01', '2021-12-31')
    streamflow_subset = streamflow_df[date_index]
    #streamflow_subset = streamflow_subset[streamflow_subset['00060_Mean_cd'] == 'A']
    streamflow_subset.dropna(subset=['00060_Mean'])
    streamflow = streamflow_subset.reset_index(drop=True)
    # I want to use the estimated values because we can't have gaps 

    sdate = date(1984, 1, 1)
    edate = date(2021, 12, 31)
    delta = edate-sdate
    total_days = delta.days + 1
    
    pct_missing = (streamflow_subset.shape[0]/total_days)*100
    
    return pct_missing


pct_miss= []
for gage_id in sbr_ref_combo_diss['GAGE_ID']:
    pct_miss.append(check_streamflow(gage_id))
sbr_ref_combo_diss['pct_stream'] = pct_miss

sbr_ref_combo_diss_100 = sbr_ref_combo_diss[sbr_ref_combo_diss['pct_stream'] == 100]

In [21]:
# dataframe with the overall A and B and the slope + pvalue for A and B 

row_list = []
for id in sbr_ref_combo_diss_100.GAGE_ID:
    print(id)
    df = create_recession_df(id, w=0.1, C=3)
    coefs_overall = calculate_coef(df, id, 'RMA')

    coefs_3r = rolling(df, id, 'RMA', 3)
    # linear regression 
    #slope_A, intercept_A, r_value_A, p_value_A, std_err_A = stats.linregress(coefs_3r.Year, coefs_3r.A)
    #slope_B, intercept_B, r_value_B, p_value_B, std_err_B = stats.linregress(coefs_3r.Year, coefs_3r.B)

    # sens slope + mann kendall 
    mks_A = mk.original_test(coefs_3r['A'])
    p_value_A = mks_A[2] # p-value 
    slope_A = mks_A[7] # Theil-Sen estimator/slope 

    mks_B = mk.original_test(coefs_3r['B'])
    p_value_B = mks_B[2] # p-value 
    slope_B = mks_B[7] # Theil-Sen estimator/slope 

    row_list.append({'gage':id, 'A':coefs_overall['A'],'B':coefs_overall['B'], 'slope_A':slope_A, 'pvalue_A':p_value_A, 'slope_B':slope_B, 'pvalue_B':p_value_B})

recession_results = pd.DataFrame(row_list)

02053800


/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['originalQ'] = streamflow['originalQ'].multiply(convert)
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['dQ'] = streamflow['originalQ'].diff()
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:112: SettingWithCopyWarning: 
A value is trying to be set on

02056900


/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['originalQ'] = streamflow['originalQ'].multiply(convert)
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['dQ'] = streamflow['originalQ'].diff()
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:112: SettingWithCopyWarning: 
A value is trying to be set on

02069700


/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['originalQ'] = streamflow['originalQ'].multiply(convert)
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['dQ'] = streamflow['originalQ'].diff()
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:112: SettingWithCopyWarning: 
A value is trying to be set on

02070000


/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['originalQ'] = streamflow['originalQ'].multiply(convert)
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_69616/409050222.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  streamflow['dQ'] = streamflow['originalQ'].diff()


ValueError: Length of values (13876) does not match length of index (13880)

In [20]:
recession_results[recession_results['pvalue_B'] <= 0.05]

,gage,A,B,slope_A,pvalue_A,slope_B,pvalue_B
0,02053800,-0.975812,1.833820,0.003227,0.177508,-0.011313,0.018452
20,03463300,-1.357076,1.801764,0.003446,0.375964,-0.014926,0.042406
23,03473000,-1.084261,2.155996,-0.002699,0.062033,0.009654,0.024611
25,03498500,-1.086449,1.920233,-0.000532,0.733463,0.008177,0.032478
26,03500000,-1.575810,2.055720,-0.000425,0.838109,-0.025441,0.004809
